In [ ]:
!pip install keras-word-char-embd
!pip install fasttext
from google.colab import drive
drive.mount('/content/gdrive')

  Created wheel for keras-word-char-embd: filename=keras_word_char_embd-0.21.0-cp36-none-any.whl size=8522 sha256=59f3edf2261b7e74ab9b9d44830bfc706dd4086ce1e0a88404770cfc7448c2fb
  Stored in directory: /root/.cache/pip/wheels/56/e1/10/a68e424326745b9ac1c0695c3da541fe6d051b129230e7456a
Successfully built keras-word-char-embd
     |████████████████████████████████| 71kB 3.4MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3095549 sha256=d07384473d6e64b66ed3af10d698f15debe72c31c002325db49653c5e7a04963
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
from keras_wc_embd import get_embedding_layer, get_dicts_generator, get_batch_input
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Embedding, Dense, LSTM, Dropout, concatenate, Input
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam,RMSprop
from keras.metrics import CategoricalCrossentropy
import fasttext as ft
from pickle import dump
import numpy as np
import random 
from keras.callbacks import Callback
from matplotlib import pyplot as plt
from IPython.display import clear_output

class PlotLearning(Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('categorical_crossentropy'))
        self.val_acc.append(logs.get('val_categorical_crossentropy'))
        self.i += 1
        f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)
        
        clear_output(wait=True)
        
        ax1.set_yscale('log')
        ax1.plot(self.x, self.losses, label="loss")
        ax1.plot(self.x, self.val_losses, label="val_loss")
        ax1.legend()
        
        ax2.plot(self.x, self.acc, label="cross entropy")
        ax2.plot(self.x, self.val_acc, label="validation cross entropy")
        ax2.legend()

        plt.show();

In [ ]:
def check_rhyme(first,second):
  first_word = normalizeWord(first)
  second_word = normalizeWord(second)
  vowels = ['a','e','i','o','u', 'à', 'è', 'é', 'ì', 'ò', 'ó', 'ù']
  #check if the two words are equal from the end to the second occured vowel
  for i in range(2, min(len(first_word), len(second_word))-1):
    if first_word[-i] != second_word[-i]:  
      return False
    if i != 1 and (first_word[-i] in vowels or second_word[-i] in vowels):
      break
  return True

def score_rhyme(first, second, enlight_end=1): 
    weight = enlight_end
    first_word = normalizeWord(first)
    second_word = normalizeWord(second)
    vowels = ['a','e','i','o','u', 'à', 'è', 'é', 'ì', 'ò', 'ó', 'ù']
    #check if the two words are equal from the end to the second occured vowel
    score = not (first_word[-1] == second_word[-1])/weight if (len(first_word) and len(second_word)) else 0
    maximum = 1
    for i in range(2, min(len(first_word), len(second_word))-1):
      if (first_word[-i] != second_word[-i]): 
        score += weight
        weight /= enlight_end
      if (first_word[-i] in vowels or second_word[-i] in vowels):
        break
    return score

#get the last verse's word normalized 
def normalizeWord(word):
    return word.strip('’)').lower()


#get a new random value from sequence
def getNewRandom(old,sequence):
  new = random.choice(sequence)
  while new == old:
    new = random.choice(sequence)
  return new

def getSyllable(word):
  vowels = ['a','e','i','o','u', 'à', 'è', 'é', 'ì', 'ò', 'ó', 'ù']
  for i in range(2, len(word)):
    if word[-i] in vowels:
      return word[-i:]
    

In [ ]:
  def generateRhymes(vocab,text,length = 30000):
    
    raw = [verse for verse in text if verse != "<tercet>"]
    uniquewords = [word for word in vocab.word_index.keys() if len(word) > 2 and word != "_unk_"]
    # print(words)
    rhymes_dict = dict()
    for word in uniquewords:
      syllable = getSyllable(word)
      rhymes_dict.setdefault(syllable,[]).append(word)

    # Remove rhyming patterns with only one word
    for rhyme in list(rhymes_dict):
      if len(rhymes_dict[rhyme]) < 2:
        rhymes_dict.pop(rhyme)

    print("The number of rhyming patterns is: ",len(rhymes_dict))
    rhymes = []
    token = "<tercet>"
    curr_syll = random.choice(list(rhymes_dict.keys()))
    print(curr_syll)
    for i in range(length):
      # Get next rhyming pattern
      next_syll = getNewRandom(curr_syll,list(rhymes_dict.keys()))
      # Fetch words according to pattern
      first = random.choice(rhymes_dict[curr_syll])
      second = random.choice(rhymes_dict[next_syll])
      third = getNewRandom(first,rhymes_dict[curr_syll])
      # Update rhyming pattern
      curr_syll = next_syll
      # Update output sequence
      rhymes.append(first)
      rhymes.append(second)
      rhymes.append(third)
      rhymes.append(token)
    return rhymes



In [ ]:
#uploaded = files.upload()

# Open the file and encode newlines as standalone symbols
with open("DC-poem-formatUTF_withoutBOL.txt", encoding='latin-1') as file:
  text = file.read().lower()
text = text.split('\n')
text = [string for string in text]
print(text)
print('Number of Characters is:', len(text))

#Initialize parameters
SEQUENCE_LEN = 3 # The length of each sequence used to predict
STEP = 1 # Stride
X = [] # Input Variables 
Y = [] # Output Variables
SEPARATOR_CHAR_TOKEN = '$'

# Fetch all the words inside the file
words = []
for verse in text:
  if verse[-1] == " ":
    verse = verse[:-1]   
  verse_split = verse.split(" ")
  words.append(verse_split[-1])
print(words)
words = words[1:]
# Fit the Tokenizer
tokenizer_word = Tokenizer(filters=[], lower=True, oov_token="_unk_")
tokenizer_word.fit_on_texts(text)
word_dict_from_idx = {v:k for k,v in tokenizer_word.word_index.items()}
word_dict_from_idx[0] = '' 

In [ ]:
dummyRhymes = generateRhymes(tokenizer_word,text,10000)
words.extend(dummyRhymes)
print(words[:100])
print(words[-100:])
words_tokenized = tokenizer_word.texts_to_sequences(words)
vocab_size = len(tokenizer_word.word_index) + 1
dump(tokenizer_word, open('generaRimaWord.pkl', 'wb'))
print('Number of Words is:', vocab_size)

tokenizer_char = Tokenizer(filters=[], lower=True, oov_token="_unk_", char_level=True)
tokenizer_char.fit_on_texts(words + [SEPARATOR_CHAR_TOKEN])
character_num = len(tokenizer_char.word_index) + 1
print('Number of Characters is:', character_num)

In [ ]:
# Flatten the resulting List
words_tokenized = [item for sublist in words_tokenized for item in sublist]

#padding_token_val = words_tokenized[0]
start_token_val = words_tokenized[0]

X = []
max_word_len = 0
print(len(words))
print(len(words_tokenized))
for i in range(3, len(words_tokenized), STEP):
  max_word_len = max([max_word_len, len(words[i-3]), len(words[i-2]), len(words[i-1])])
  X.append([words_tokenized[i-3],words_tokenized[i-2],words_tokenized[i-1]])
print('Number of Sequences:', len(X), len(Y))

Y = []
for i in range(3, len(words_tokenized), STEP):
  Y.append(words_tokenized[i])

X = np.array(X)
Y = np.array(Y)
print(X[:10])
Y = to_categorical(Y, num_classes=vocab_size)

In [ ]:
from keras.losses import categorical_crossentropy
from tensorflow.math import add, reduce_sum
from tensorflow import constant, float32
import tensorflow
#Loss
from keras.backend import argmax, get_value
from tensorflow import make_ndarray, make_tensor_proto 

def convert_tensor_to_list_of_words(tensor):
  return [word_dict_from_idx[idx] for idx in make_ndarray(make_tensor_proto(argmax(tensor)))]

def custom_loss(y_true, y_pred):
  rhyme_weight = 10
  true = convert_tensor_to_list_of_words(y_true)
  pred = convert_tensor_to_list_of_words(y_pred)
  rhymes_scores = [rhyme_weight*score_rhyme(t, p, enlight_end=1) for t,p in zip(true, pred)]
  s = constant(rhymes_scores, dtype=float32)
  cat_loss = categorical_crossentropy(y_true, y_pred, from_logits=False)
  # tensorflow.print(cat_loss.size)
  return add(s, cat_loss) 

In [ ]:
# import tensorflow as tf

# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
ftmodel = ft.train_unsupervised(input='DC-poem-formatUTF_withoutBOL.txt', minn=3, maxn=6, dim=100, epoch=20)
embedding_dim = 100
def build_embedding_matrix(vocabulary,embedding_dimension):
  numWords = len(vocabulary.word_index) + 1
  embedding = np.empty(shape=(numWords,embedding_dimension))
  for word in vocabulary.word_index:
      embedding[vocabulary.word_index[word]] = ftmodel.get_word_vector(word)
  return embedding
embedding_matrix = build_embedding_matrix(tokenizer_word,embedding_dim)

In [ ]:
print(X.shape)

In [ ]:
# Create the neural network
model_filepath="/content/gdrive/My Drive/generaword/weights-{epoch:02d}-{loss:.4f}.hdf5"
input_words = Input(shape=X.shape[1])
embedding_words = Embedding(vocab_size, embedding_dim,input_length=X.shape[1], mask_zero=True,weights = [embedding_matrix],trainable=False)(input_words)
layer1 = LSTM(128, dropout=0.2)(embedding_words)
dense_softmax = Dense(vocab_size, activation='softmax')(layer1)
model = Model(input_words, dense_softmax)
print(model.summary())

In [ ]:
from tensorflow.keras.models import load_model

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'], run_eagerly=True)
# Define Callbacks
filename = "/content/model-GeneraRimaWord60Epoch.h5"
model = load_model(filename,custom_objects={'custom_loss': custom_loss})
model.load_weights(filename)
es = EarlyStopping(monitor=custom_loss, mode='min', verbose=2, patience=150)
ck = ModelCheckpoint(model_filepath, monitor=custom_loss, verbose=1, mode='min')
# Start Training
model.fit(X, Y,shuffle=False, epochs=200, callbacks=[es,ck,PlotLearning()])
model.save('/content/gdrive/My Drive/Dante_model/model.h5')

model.save('model-last.h5')



In [ ]:
model.save('model-GeneraRimaWord180Epoch.h5')

In [ ]:
model.compile(loss=custom_loss, optimizer='adam', metrics=['categorical_crossentropy'], run_eagerly=True)
print(model.summary())
# Define Callbacks
es = EarlyStopping(monitor=custom_loss, mode='min', verbose=2, patience=150)

ck = ModelCheckpoint(model_filepath, monitor=custom_loss, verbose=1, mode='min')
model.fit([X_word, X_char], Y, batch_size=128, epochs=50, callbacks=[ck,PlotLearning()], validation_split = 0)

In [ ]:
model_loaded = model

In [ ]:
from pickle import load
from keras.models import load_model
import numpy as np 
def sample(preds, temperature=1.0):
  # helper function to sample an index from a probability array
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

# # Initialize variables
n_words = 100
predicted_word = ''
generate = []

# Load pre-trained data
tokenizer_word = load(open('/content/generaRimaWord.pkl', 'rb'))
#model_loaded = load_model('/content/100epocheGeneraRimaWord.h5', custom_objects={'custom_loss': custom_loss})
# Encode initial input
previous = ['<start>', 'vita', 'oscura']
previous = [tokenizer_word.word_index[word] for word in previous]
print(previous)
#while predicted_word != '_end_':
for i in range(n_words):
  prediction = model.predict([previous], verbose=0)
  predicted_word = tokenizer_word.sequences_to_texts([[np.argmax(prediction)]])
  previous.append(tokenizer_word.word_index[predicted_word[0]])
  previous = previous[-3:]
  generate.append(predicted_word)
  print(predicted_word)
# Format and print the result
generated = ['\n' if x=='_verse_' or x=='_end_' else x for x in generated]
output = ' '.join(generated)
print(output)
#with open("output/seq50epoch100.txt", "w") as file:
#	file.write(output)